In [1]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher, expand_level3
from heterogt.utils.train import train_with_early_stopping, PHENO_ORDER
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTFineTune

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-IV",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"],
    task_index = 3,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    # only for tf layer
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}], 
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    use_pretrained_model = True,
    pretrain_mask_rate = 0.7,
    pretrain_cls_ontology_weight = 5e-2,
    pretrain_visit_ontology_weight = 5e-2,
    pretrain_adm_type_weight = 0,
    dec_loss_lambda = 1e-2,
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: next_diag_6m


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = len(PHENO_ORDER)  # a set of predifined diseases
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 20
Group code vocabulary size: 19


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 3.7640246109301487 %
Percentage of READMISSION in test dataset: 79.43057063578236 %
Percentage of STAY>7 days in test dataset: 35.070575461454936 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                 max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)

In [10]:
long_adm_seq_crite = 3
val_long_seq_idx, test_long_seq_idx = [], []
for i in range(len(val_dataset)):
    hadm_id = list(val_dataset.records.keys())[i]
    num_adms = len(val_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        val_long_seq_idx.append(i)
for i in range(len(test_dataset)):
    hadm_id = list(test_dataset.records.keys())[i]
    num_adms = len(test_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        test_long_seq_idx.append(i)
print(len(val_long_seq_idx), len(test_long_seq_idx))

686 773


In [11]:
num_group_code = []
for i in range(len(train_dataset)):
    input_ids, token_types, adm_index, age_ids, diag_group_codes, labels = train_dataset[i]
    count = (token_types[0] == 6).sum().item()
    num_group_code.append(count)
print("Mean group token numer per patient", np.mean(num_group_code))

Mean group token numer per patient 0.8440647482014388


In [12]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [13]:
for batch in train_dataloader:
    pass  # just to check if the dataloader works
for batch in val_dataloader:
    pass  # just to check if the dataloader works
for batch in test_dataloader:
    pass  # just to check if the dataloader works
print("All pass!")

All pass!


In [14]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "f1"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "f1"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [15]:
input_ids, token_types, adm_index, age_ids, diag_code_group_dicts, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age IDs shape:", age_ids.shape)
print("Diag Code Group Dict number:", len(diag_code_group_dicts))
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 139])
Token Types shape: torch.Size([32, 139])
Admission Index shape: torch.Size([32, 139])
Age IDs shape: torch.Size([32, 4])
Diag Code Group Dict number: 32
Labels shape: torch.Size([32, 18])


# Model Walkthrough

In [16]:
# load pretrained model
if config.use_pretrained_model:
    pretrain_exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.pretrain_cls_ontology_weight}-{config.pretrain_visit_ontology_weight}-{config.pretrain_adm_type_weight}"
)
    print(pretrain_exp_name)
    save_path = "./pretrained_models/" + pretrain_exp_name
    state_dict = torch.load(f"{save_path}/pretrained_model.pt", map_location="cpu")

MIMIC-IV-0.7-64-0.05-0.05-0


In [17]:
final_metrics, final_long_seq_metrics = [], []
for i in range(15):
    print(f"================================{i+1}==================================")
    model = HeteroGTFineTune(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
    if config.use_pretrained_model:
        model.load_weight(state_dict)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric, best_test_long_seq_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                                                            optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                                                            dec_loss_lambda=config.dec_loss_lambda, 
                                                                            val_long_seq_idx=val_long_seq_idx, test_long_seq_idx=test_long_seq_idx, 
                                                                            eval_metric=eval_metric, return_model=False)
    final_metrics.append(best_test_metric)
    final_long_seq_metrics.append(best_test_long_seq_metric)

================================1==================================


Running inference: 100%|██████████| 130/130 [00:05<00:00, 23.48it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure         100.0000   0.1029   0.2055   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         61.7617  53.0249  57.0608   
Chronic kidney disease                       80.9762  69.7198  74.9276   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    75.1131  68.7371  71.7838   
Coronary atherosclerosis and related         67.3874  75.5556  71.2381   
Disorders of lipid metabolism                62.7356  85.0033  72.1913   
Essential hypertension                       72.9847  77.3672  75.1121   
Fluid and electrolyte disorders              59.2593   2.3827   4.5812   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 132/132 [00:05<00:00, 23.68it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          66.6667   0.2035   0.4057   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         61.7952  48.7738  54.5178   
Chronic kidney disease                       84.9704  72.3061  78.1284   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    71.6285  63.6158  67.3848   
Coronary atherosclerosis and related         74.3925  75.6654  75.0236   
Disorders of lipid metabolism                63.3588  85.0192  72.6080   
Essential hypertension                       75.3154  78.5020  76.8757   
Fluid and electrolyte disorders              68.9655   4.3478   8.1800   
Gastrointestinal hemorrhage           

...

                                           precision   recall       f1  \
Acute and unspecified renal failure          40.7725  38.6572  39.6867   
Acute cerebrovascular disease                16.6667   3.5714   5.8824   
Acute myocardial infarction                  32.4324   8.3333  13.2597   
Cardiac dysrhythmias                         72.5027  61.3079  66.4370   
Chronic kidney disease                       74.6124  77.5428  76.0494   
Chronic obstructive pulmonary disease        62.8399  59.7701  61.2666   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    71.3548  62.4859  66.6265   
Coronary atherosclerosis and related         79.8736  72.0532  75.7621   
Disorders of lipid metabolism                72.4324  77.2087  74.7443   
Essential hypertension                       79.8687  62.6072  70.1923   
Fluid and electrolyte disorders              44.4345  35.8696  39.6953   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 130/130 [00:05<00:00, 22.98it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          43.9883  30.8642  36.2757   
Acute cerebrovascular disease                25.0000   2.7778   5.0000   
Acute myocardial infarction                  23.6842   5.7325   9.2308   
Cardiac dysrhythmias                         73.1423  61.2989  66.6989   
Chronic kidney disease                       71.5897  75.2155  73.3579   
Chronic obstructive pulmonary disease        58.3596  53.7791  55.9758   
Conduction disorders                         14.2857   2.1978   3.8095   
Congestive heart failure; nonhypertensive    68.5315  71.0145  69.7509   
Coronary atherosclerosis and related         71.3144  81.1111  75.8979   
Disorders of lipid metabolism                72.6807  74.3942  73.5275   
Essential hypertension                       73.3817  70.0346  71.6691   
Fluid and electrolyte disorders              42.9422  37.6024  40.0953   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 132/132 [00:05<00:00, 23.08it/s]

                                           precision   recall       f1  \
Acute and unspecified renal failure          41.7367  30.3154  35.1208   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                  26.8293   7.6389  11.8919   
Cardiac dysrhythmias                         72.1088  57.7657  64.1452   
Chronic kidney disease                       75.6784  75.8308  75.7545   
Chronic obstructive pulmonary disease        64.3312  58.0460  61.0272   
Conduction disorders                         14.2857   2.7778   4.6512   
Congestive heart failure; nonhypertensive    66.6667  70.2825  68.4268   
Coronary atherosclerosis and related         77.1020  81.9392  79.4470   
Disorders of lipid metabolism                72.9816  73.4955  73.2376   
Essential hypertension                       76.9323  73.4134  75.1317   
Fluid and electrolyte disorders              42.3904  38.5507  40.3795   
Gastrointestinal hemorrhage           

In [18]:
def topk_avg_performance_formatted(performances, long_seq_performances, k=5):
    metrics = ["f1", "auc", "prauc"]
    scores = {m: np.array([p[m] for p in performances]) for m in metrics}

    # 计算排名（值越大排名越靠前）
    ranks = {m: (-scores[m]).argsort().argsort() + 1 for m in metrics}
    avg_ranks = np.mean(np.stack([ranks[m] for m in metrics], axis=1), axis=1)

    # 选 top-k
    topk_idx = np.argsort(avg_ranks)[:k]
    final_avg = {m: np.mean([performances[i][m] for i in topk_idx]) for m in performances[0].keys()}
    final_std = {m: np.std([performances[i][m] for i in topk_idx], ddof=0) for m in performances[0].keys()}
    final_long_seq_avg = {m: np.mean([long_seq_performances[i][m] for i in topk_idx]) for m in long_seq_performances[0].keys()}
    final_long_seq_std = {m: np.std([long_seq_performances[i][m] for i in topk_idx], ddof=0) for m in long_seq_performances[0].keys()}

    # 打印结果（转百分比，均保留两位小数）
    print("Final Metrics:")
    for m in performances[0].keys():
        mean_val = final_avg[m]
        std_val = final_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")
    print("\nFinal Long Sequence Metrics:")
    for m in long_seq_performances[0].keys():
        mean_val = final_long_seq_avg[m]
        std_val = final_long_seq_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")

In [19]:
topk_avg_performance_formatted(final_metrics, final_long_seq_metrics)

Final Metrics:
precision: 45.34 ± 0.85
recall: 37.98 ± 0.92
f1: 39.62 ± 0.41
auc: 78.54 ± 0.25
prauc: 44.64 ± 0.16

Final Long Sequence Metrics:
precision: 42.78 ± 0.97
recall: 39.15 ± 1.08
f1: 40.01 ± 0.50
auc: 77.56 ± 0.64
prauc: 46.31 ± 1.09
